In [48]:
%run init_notebook.py
from settings import *

In [127]:
import seaborn as sns
import sympy as sp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import statsmodels.api as sm

In [128]:
df = sns.load_dataset('iris')

#### Regression

In [129]:
a = np.random.randint(0, 10, size=(1, 20))

In [130]:
a

array([[3, 9, 8, 7, 1, 6, 2, 3, 8, 3, 0, 7, 8, 6, 5, 8, 3, 9, 8, 1]])

In [131]:
a @ a.transpose()

array([[719]])

In [132]:
params = sp.symbols(['X', 'y', 'n', 'k'])
X, y, n, k = params

In [133]:
X = sp.MatrixSymbol('X', n, k)
b = sp.MatrixSymbol('b', k, 1)
y = sp.MatrixSymbol('y', n, 1)
e = sp.MatrixSymbol('epsilon', n, 1)

β_h = (X.transpose() @ X).inverse() * (X.transpose() @ y)
σ2_h = (e.transpose() @ e) / (n-k)
# β_cov = σ2_h * (X.transpose() @ X).inverse()

In [134]:
σ2_h

1/(-k + n)*epsilon.T*epsilon

In [135]:
β_h

(X.T*X)**(-1)*X.T*y

In [136]:
# normalise
def normalise(arr):
    return (arr - arr.min())/(arr.max() - arr.min())

In [137]:
y = df['petal_width'].values.reshape(-1, 1)
X = df.drop(['petal_width', 'species'], axis=1).values.reshape(-1, 3)

# normalise
for i in range(0, X.shape[1]):
    X[:, i] = normalise(X[:, i])
    
# add constant
X = np.concatenate((np.full((X.shape[0], 1), 1), X), axis=1)


In [173]:
class OLS_reg:
    
    def __init__(self,
                 X: np.array,
                 y: np.array,
                 critical_multicol_val: float = .85):
            self.X = X
            self.y = y
            
            # check dimensions
            self.n = X.shape[0]
            self.k = X.shape[1]
            
            self.critical_multicol_val = critical_multicol_val
            
            self.beta_hat = None
            self.y_hat = None
            self.resid = None
            self.sigma2_h = None
            self.beta_hat_cov = None
            self.beta_standarerr = None
            self.X_corrm = None
            self.r2 = None
            
            self._check_assumptions()
            pass
    def _check_assumptions(self):
        # dimensions
        assert self.n / self.k >= 5, 'Dimensionality ratio n/k too small'
        
        # multicolineratiry
        self.X_corrm = np.corrcoef(X.reshape(self.k, -1))
        self.X_corrm = np.abs(self.X_corrm) > self.critical_multicol_val
        np.sum(np.abs(np.corrcoef(X.reshape(4, -1))) > .85)
        assert np.sum(self.X_corrm) == self.k, f'Multicolinerarity detected {self.X_corrm}'
        pass
    
    def check_gauss_markov(self):
        
        # linear relationship
        
        # multicolinearity assumption
        
        # zero conditional mean
        
        # homoskedacity & no autocorrelation
        
        # central limit theorem
        pass
    
    def _get_metrics(self):
        self.r2 = 1 - np.sum((self.y.reshape(-1) - self.y_hat)**2) / np.sum((self.y.reshape(-1) - self.y.mean())**2)
        
        
    def fit(self):
        # beta hat
        self.beta_hat = np.linalg.inv(self.X.transpose() @ self.X) @ (self.X.transpose() @ self.y)
        
        # y hat
        self.y_hat = (self.X @ self.beta_hat).reshape(-1)
        # residuals
        self.resid = self.y.reshape(-1) - self.y_hat
        # resid variance
        self.sigma2_h = (self.resid.transpose() @ self.resid) / (self.n - self.k)
        # resid cov matrix
        self.beta_hat_cov = self.sigma2_h * np.linalg.inv(self.X.transpose() @ self.X)
        # resid standard errors
        self.beta_standarerr = np.sqrt(self.beta_hat_cov.diagonal())
        pass
    
    def summary(self):
        self._get_metrics()
        df = pd.DataFrame(columns=[f'X{i}' for i in range(0, self.k)],
                          index=['beta_hat', 'beta_std_err'],
                          data=[self.beta_hat.reshape(-1),
                                self.beta_standarerr])
        
        df_stats = pd.DataFrame(columns=['r2'],
                                index=[1],
                                data=[self.r2])
        return df, df_stats

In [174]:
mymodel = OLS_reg(X, y)
mymodel.fit()

In [175]:
mymodel.beta_hat.reshape(-1)

array([-0.1618113 , -0.74615787,  0.53478851,  3.09209038])

In [176]:
mymodel.beta_standarerr

array([0.0639615 , 0.17102223, 0.1174513 , 0.14449893])

In [177]:
mymodel.summary()

(                    X0        X1        X2        X3
 beta_hat     -0.161811 -0.746158  0.534789  3.092090
 beta_std_err  0.063961  0.171022  0.117451  0.144499,
         r2
 1  0.93785)

In [75]:
model = sm.OLS(y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.938
Model:                            OLS   Adj. R-squared:                  0.937
Method:                 Least Squares   F-statistic:                     734.4
Date:                Sat, 19 Nov 2022   Prob (F-statistic):           7.83e-88
Time:                        15:33:13   Log-Likelihood:                 36.751
No. Observations:                 150   AIC:                            -65.50
Df Residuals:                     146   BIC:                            -53.46
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1618      0.064     -2.530      0.012      -0.288      -0.035
x1            -0.7462      0.171     -4.363      0.000      -1.084      -0.408
x2             0.5348      0.117      4.553      0.000       0.303       0.767
x3             3.0921      0.144     21.399      0.000       2.807       3.378
==============================================================================
Omnibus:                        5.609   Durbin-Watson:                   1.573
Prob(Omnibus):                  0.061   Jarque-Bera (JB):                6.811
Skew:                           0.223   Prob(JB):                       0.0332
Kurtosis:                       3.944   Cond. No.                         19.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [191]:
mymodel.beta_hat.reshape(-1)

array([-0.1618113 , -0.74615787,  0.53478851,  3.09209038])

In [192]:
mymodel.beta_stand_err

array([0.00409107, 0.0292486 , 0.01379481, 0.02087994])